In [1]:
import pandas as pd
from pathlib import Path

In [2]:
data = Path('../data')

## EDA

In [3]:
list(data.glob('*.csv'))

[WindowsPath('../data/customers.csv'),
 WindowsPath('../data/orders.csv'),
 WindowsPath('../data/products.csv'),
 WindowsPath('../data/sales.csv')]

In [5]:
df_sales = pd.read_csv(data / 'sales.csv')
df_sales.head()

,sales_id,order_id,product_id,price_per_unit,quantity,total_price
0,0,1,218,106,2,212
1,1,1,481,118,1,118
2,2,1,2,96,3,288
3,3,1,1002,106,2,212
4,4,1,691,113,3,339


In [8]:
len(df_sales)

5000

In [10]:
df_orders = pd.read_csv(data / 'orders.csv')
df_orders.head()

,order_id,customer_id,payment,order_date,delivery_date
0,1,64,30811,2021-8-30,2021-09-24
1,2,473,50490,2021-2-3,2021-02-13
2,3,774,46763,2021-10-8,2021-11-03
3,4,433,39782,2021-5-6,2021-05-19
4,5,441,14719,2021-3-23,2021-03-24


In [11]:
len(df_orders)

1000

In [14]:
df_customers = pd.read_csv(data / 'customers.csv')
df_customers.head()

,customer_id,customer_name,gender,age,home_address,zip_code,city,state,country
0,1,Leanna Busson,Female,30,8606 Victoria TerraceSuite 560,5464,Johnstonhaven,Northern Territory,Australia
1,2,Zabrina Harrowsmith,Genderfluid,69,8327 Kirlin SummitApt. 461,8223,New Zacharyfort,South Australia,Australia
2,3,Shina Dullaghan,Polygender,59,269 Gemma SummitSuite 109,5661,Aliburgh,Australian Capital Territory,Australia
3,4,Hewet McVitie,Bigender,67,743 Bailey GroveSuite 141,1729,South Justinhaven,Queensland,Australia
4,5,Rubia Ashleigh,Polygender,30,48 Hyatt ManorSuite 375,4032,Griffithsshire,Queensland,Australia


In [15]:
len(df_customers)

1000

In [17]:
df_products = pd.read_csv(data / 'products.csv')
df_products.head()

,product_ID,product_type,product_name,size,colour,price,quantity,description
0,0,Shirt,Oxford Cloth,XS,red,114,66,"A red coloured, XS sized, Oxford Cloth Shirt"
1,1,Shirt,Oxford Cloth,S,red,114,53,"A red coloured, S sized, Oxford Cloth Shirt"
2,2,Shirt,Oxford Cloth,M,red,114,54,"A red coloured, M sized, Oxford Cloth Shirt"
3,3,Shirt,Oxford Cloth,L,red,114,69,"A red coloured, L sized, Oxford Cloth Shirt"
4,4,Shirt,Oxford Cloth,XL,red,114,47,"A red coloured, XL sized, Oxford Cloth Shirt"


In [18]:
len(df_products)

1260

We need to get customer id - we do it by joining it with the orders table

In [22]:
df_sales = df_sales.merge(df_orders[['order_id', 'customer_id']])

In [23]:
df_sales.head()

,sales_id,order_id,product_id,price_per_unit,quantity,total_price,customer_id
0,0,1,218,106,2,212,64
1,1,1,481,118,1,118,64
2,2,1,2,96,3,288,64
3,3,1,1002,106,2,212,64
4,4,1,691,113,3,339,64


Now let's check the sparsity of our interactions

In [28]:
n_items = df_sales.product_id.nunique()
n_users = df_sales.customer_id.nunique()
n_interactions = len(df_sales)

print(n_items, n_users, n_interactions)
n_interactions / (n_users * n_items)

1233 616 5000


0.006583035780116072

We will have a very sparse matrix

## Validation

Let's set aside some data for testing 

In [33]:
df_sales.order_id.unique()[[0, 1, 2, -3, -2, -1]]

array([  1,   2,   3, 997, 998, 999], dtype=int64)

In [34]:
df_train = df_sales[df_sales.order_id <= 900]
df_val = df_sales[df_sales.order_id > 900]


We need to have a good baseline. Let's use the most frequest items

In [39]:
df_train.product_id.value_counts().iloc[:10]

740     12
579     11
78      11
843     10
1188    10
968     10
1177     9
1038     9
182      9
757      9
Name: product_id, dtype: int64

We will have the following recommendation scenario: we suggest 10 products and see how many
of them the user will click on.

We'll use a simple evaluation technique: we'll calculate how many suggested items the user actually bought. 

It's called "Average presicion" ([link](https://sdsawtelle.github.io/blog/output/mean-average-precision-MAP-for-recommender-systems.html))

In [63]:
n_rec = 10
freq = df_train.product_id.value_counts().iloc[:n_rec].index.values
freq

array([ 740,  579,   78,  843, 1188,  968, 1177, 1038,  182,  757],
      dtype=int64)

In [64]:
import numpy as np

In [65]:
val_ground_truth = df_val.groupby('order_id').product_id.apply(set)
val_ground_truth

order_id
901                     {650, 207, 1144, 441, 927}
902                  {683, 365, 21, 118, 568, 892}
903                {992, 420, 516, 651, 1132, 121}
904                   {1096, 1098, 148, 150, 1240}
905                                 {870, 486, 79}
                          ...                     
995                  {837, 756, 564, 244, 536, 24}
996    {1034, 381, 1011, 53, 1144, 794, 379, 1053}
997                                     {514, 509}
998                  {321, 66, 972, 251, 541, 478}
999                               {872, 1105, 998}
Name: product_id, Length: 99, dtype: object

In [66]:
n_val = len(val_ground_truth)
recommendations = np.repeat([freq], n_val, axis=0)
recommendations[:10]

array([[ 740,  579,   78,  843, 1188,  968, 1177, 1038,  182,  757],
       [ 740,  579,   78,  843, 1188,  968, 1177, 1038,  182,  757],
       [ 740,  579,   78,  843, 1188,  968, 1177, 1038,  182,  757],
       [ 740,  579,   78,  843, 1188,  968, 1177, 1038,  182,  757],
       [ 740,  579,   78,  843, 1188,  968, 1177, 1038,  182,  757],
       [ 740,  579,   78,  843, 1188,  968, 1177, 1038,  182,  757],
       [ 740,  579,   78,  843, 1188,  968, 1177, 1038,  182,  757],
       [ 740,  579,   78,  843, 1188,  968, 1177, 1038,  182,  757],
       [ 740,  579,   78,  843, 1188,  968, 1177, 1038,  182,  757],
       [ 740,  579,   78,  843, 1188,  968, 1177, 1038,  182,  757]],
      dtype=int64)

In [69]:
average_precision = 0

for i, gt in enumerate(val_ground_truth):
    cnt = 0
    for item_id in recommendations[i]:
        if item_id in gt:
            cnt = cnt + 1
    precision = cnt / n_rec
    average_precision = average_precision + precision

average_precision = average_precision / n_val
average_precision

0.005050505050505051

That's our baseline. Let's see if we can improve it with collaborative filtering